In [33]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import pandas as pd
import torch
from rouge_score import rouge_scorer
from tqdm import tqdm

In [ ]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)

In [36]:
train_df = pd.read_csv('data/train.csv', dtype={'text': str, 'titles': str})
validation_df = pd.read_csv('data/validation.csv', dtype={'text': str, 'titles': str})
# longest title is 967 words

In [ ]:
example_article = "Sur les réseaux sociaux, les images sont impressionnantes. Dimanche matin à Venise, l'équipage du MSC Opéra a perdu le contrôle du paquebot, à son arrivée dans le port de la cité des Doges. Le navire, qui peut contenir plus de 2.600 passagers, est venu heurter le quai auquel il voulait s'arrimer. Le paquebot a raclé le quai sur plusieurs mètres, suscitant la panique des personnes à terre, avant de percuter un autre bateau touristique, le Michelangelo, stoppant ainsi sa course. Des témoins ont filmé la scène. Les vidéos montrent des touristes courant pour tenter de fuir le paquebot, qui ne semble pas vouloir s'arrêter. Quatre personnes ont été blessées dans cet accident : deux légèrement, tandis que les deux autres ont été transportées à l'hôpital pour des examens. L'incident s'est produit à San Basilio-Zaterre, dans le canal de la Giudecca, où de nombreux navires de croisière s'arrêtent pour permettre à leurs passagers de visiter Venise.Selon le quotidien italien Corriere della Serra, cette course folle serait due aux forts courants et à la rupture de l'un des câbles qui reliait le navire au remorqueur, qui l'aidait à entrer dans le canal."

example_title = 'Le bateau de croisière, long de 275 m, a percuté un quai lors de son arrivée dans le port de Venise, dimanche 2 juin. Quatre personnes ont été blessées.'

In [48]:
def mbart_summary(text_series: pd.Series, batch_size: int = 4) -> pd.Series:
    summaries = []
    tokenizer.src_lang = "fr_XX"
    assert isinstance(model, MBartForConditionalGeneration)

    for i in range(0, len(text_series), batch_size):
        batch = text_series[i:i+batch_size].tolist()
        encoded_articles = tokenizer(
            batch, return_tensors="pt", padding=True, truncation=True, max_length=1024)
        summary_tokens = model.generate(
            **encoded_articles, max_length=150, num_beams=4, early_stopping=True)

        batch_summaries = [tokenizer.decode(
            g, skip_special_tokens=True) for g in summary_tokens]
        summaries.extend(batch_summaries)

    return pd.Series(summaries)

In [49]:
def score_summaries(predicted_summary: pd.Series, reference_summary: pd.Series):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = []
    for i in tqdm(range(len(predicted_summary))):
        score = scorer.score(predicted_summary[i], reference_summary[i])[
            'rougeL'][2]
        scores.append(score)
    avg_score = sum(scores) / len(scores)

    return avg_score

In [51]:
# summaries = mbart_summary(validation_df['text'][:10])

# time mbart_summary with batch_size=1, 2, 4, 8

import time
for batch_size in [1, 2, 4, 8]:
    start = time.time()
    summaries = mbart_summary(validation_df['text'][:8], batch_size=batch_size)
    end = time.time()
    print(f"batch_size={batch_size} took {end-start} seconds")

batch_size=1 took 219.00986194610596 seconds
batch_size=2 took 139.4237277507782 seconds
batch_size=4 took 112.87465715408325 seconds
batch_size=8 took 131.28534698486328 seconds


In [ ]:
score_summaries(summaries, validation_df['titles'][:10])

100%|██████████| 10/10 [00:00<00:00, 156.44it/s]


0.11877055351518588